In [1]:
import pandas as pd
import numpy as np
from scipy import sparse as sps
import nltk
import pickle
from sklearn.feature_extraction.text import TfidfVectorizer
import matplotlib.pyplot as plt

In [2]:
import lenskit
import lenskit.crossfold as xf
from  lenskit.crossfold import TTPair

In [3]:
from lenskit.algorithms import als, basic, item_knn, user_knn
from lenskit.algorithms.basic import Fallback
from lenskit.algorithms.als import BiasedMF, ImplicitMF
from lenskit.algorithms.implicit import BPR

In [4]:
from lenskit.batch import MultiEval
from lenskit.crossfold import partition_users, SampleN
from lenskit import batch, topn, util
from tf_idf import tf_idf
from LDA import LDA
from LDA_sklearn import LDA_SK

In [5]:
saved = open("pickle/game_reviews.pickle","rb")
game_reviews = pickle.load(saved)
game_reviews.head()

,funny,helpful,item_id,last_edited,posted,recommend,review,user_id
0,,No ratings yet,1250,,"Posted November 5, 2011.",True,Simple yet with great replayability. In my opi...,76561197970982479
1,,No ratings yet,22200,,"Posted July 15, 2011.",True,It's unique and worth a playthrough.,76561197970982479
2,,No ratings yet,43110,,"Posted April 21, 2011.",True,Great atmosphere. The gunplay can be a bit chu...,76561197970982479
3,,15 of 20 people (75%) found this review helpful,251610,,"Posted June 24, 2014.",True,I know what you think when you see this title ...,js41637
4,,0 of 1 people (0%) found this review helpful,227300,,"Posted September 8, 2013.",True,For a simple (it's actually not all that simpl...,js41637


In [6]:
user_game = open("pickle/user_games.pickle","rb")
user_games = pickle.load(user_game)
user_games.head()

,user_id,item_id,item_name
0,76561197970982479,10,Counter-Strike
1,76561197970982479,20,Team Fortress Classic
2,76561197970982479,30,Day of Defeat
3,76561197970982479,40,Deathmatch Classic
4,76561197970982479,50,Half-Life: Opposing Force


In [7]:
user_games = user_games.rename(columns={'user_id': 'user', 'item_id': 'item'})
user_games_list = user_games[['item', 'user']]
user_games_list.head()

,item,user
0,10,76561197970982479
1,20,76561197970982479
2,30,76561197970982479
3,40,76561197970982479
4,50,76561197970982479


In [8]:
reviews = game_reviews[['item_id', 'user_id','review']]
reviews = reviews.rename(columns={'user_id': 'user', 'item_id': 'item'})
reviews.shape

(58430, 3)

In [9]:
def groupby_count(df, group, count):
    game_count = pd.DataFrame()
    game_count['count'] = df.groupby(group)[count].nunique()
    return game_count

In [10]:
def prune(df, condition):     ## returns a dataframe that meet the given condition
    user_n = df.loc[df['count'] < condition ]
    return user_n

In [11]:
game_count = groupby_count(reviews, 'user', 'item')
game_count.sort_values(by = 'count', ascending = False).head()

,count
user,
76561198045431856,10
chicken_tonight,10
76561198048348337,10
registeredso,10
ItsDerRey,10


In [12]:
user_5 = prune(game_count, 5)
user_less_5 = user_5.index
pruned_data_5 = reviews.set_index('user').drop(user_less_5)
pruned_data_5.reset_index(inplace = True)
pruned_data_5.head()

,user,item,review
0,evcentric,248820,A suitably punishing roguelike platformer. Wi...
1,evcentric,370360,"""Run for fun? What the hell kind of fun is that?"""
2,evcentric,237930,"Elegant integration of gameplay, story, world ..."
3,evcentric,263360,"Random drops and random quests, with stat poin..."
4,evcentric,107200,Fun balance of tactics and strategy. Potentia...


In [19]:
%%time
lda = LDA_SK()
lda.fit(pruned_data_5)
lda.predict_for_user('evcentric',[10,20,30,40,50,60])

CPU times: user 2min 20s, sys: 6.74 s, total: 2min 26s
Wall time: 1min 31s


In [14]:
%%time
tf = tf_idf()
tf.fit(pruned_data_5)
tf.predict_for_user('evcentric',[10,20,30,40,50,60])

CPU times: user 1min 13s, sys: 5.75 s, total: 1min 19s
Wall time: 1min 18s


In [15]:
%%time
lda_obj = LDA()
lda_obj.fit(pruned_data_5)
lda_obj.predict_for_user('evcentric',[10,20,30,40,50,60])

CPU times: user 3min 41s, sys: 7.73 s, total: 3min 48s
Wall time: 1min 43s


In [16]:
lda_obj.similarity_matrix

array([[0.40799374, 0.17206098, 0.27795042, ..., 0.04953522, 0.16131019,
        0.09823617],
       [0.17206098, 0.4874497 , 0.18401638, ..., 0.0497546 , 0.07845057,
        0.02350429],
       [0.27795042, 0.18401638, 0.47518824, ..., 0.04945513, 0.48939702,
        0.48185635],
       ...,
       [0.04953522, 0.0497546 , 0.04945513, ..., 0.05      , 0.04978469,
        0.0488125 ],
       [0.16131019, 0.07845057, 0.48939702, ..., 0.04978469, 0.71937967,
        0.81250285],
       [0.09823617, 0.02350429, 0.48185635, ..., 0.0488125 , 0.81250285,
        0.95306405]])

In [17]:
lda.similarity_matrix

array([[2.97985293e-01, 2.55876099e-01, 1.94852639e-01, ...,
        5.00000000e-02, 1.45067993e-01, 2.72702164e-01],
       [2.55876099e-01, 5.06208256e-01, 7.30040498e-02, ...,
        5.00000000e-02, 5.44515231e-04, 1.54907978e-03],
       [1.94852639e-01, 7.30040498e-02, 3.42253071e-01, ...,
        5.00000000e-02, 2.29528462e-01, 4.30663917e-01],
       ...,
       [5.00000000e-02, 5.00000000e-02, 5.00000000e-02, ...,
        5.00000000e-02, 5.00000000e-02, 5.00000000e-02],
       [1.45067993e-01, 5.44515231e-04, 2.29528462e-01, ...,
        5.00000000e-02, 4.14980531e-01, 5.08403101e-01],
       [2.72702164e-01, 1.54907978e-03, 4.30663917e-01, ...,
        5.00000000e-02, 5.08403101e-01, 9.53093750e-01]])

In [18]:
tf.similarity_matrix

array([[1.        , 0.1499391 , 0.13796804, ..., 0.        , 0.117523  ,
        0.07885343],
       [0.1499391 , 1.        , 0.06882521, ..., 0.        , 0.07033395,
        0.04977039],
       [0.13796804, 0.06882521, 1.        , ..., 0.        , 0.04951468,
        0.04510642],
       ...,
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.117523  , 0.07033395, 0.04951468, ..., 0.        , 1.        ,
        0.06608514],
       [0.07885343, 0.04977039, 0.04510642, ..., 0.        , 0.06608514,
        1.        ]])